# Introducción

En esta notebook se llevarán a cabo los pasos para la implementación de un modelo que pueda detectar si un mensaje es spam o no.
Esta dividido por titulos con las acciones a seguir para la resolución del problema.

# Imports

* Pandas nos ayudara para la obtencion de los datos en fuentes externas.
* De sklearn vamos a usar:
  1. train_test_split para entrenar y verificar un modelo.
  2. metrics para calcular la precisión del modelo.
  3. CountVectorizer para obtener una representación numérica de las palabras en los mensajes que se probaran en el algoritmo.
  4. MultinomialNB como algoritmo para el modelo (se utiliza Multinomial ya que se es util para la clasificación de texto).
* Drive de google para poder utilizar archivos de un Drive. 





In [1]:
import pandas as pd   #Para leer el csv
from sklearn.model_selection import train_test_split  #Separar en conjuntos de entrenamiento y validacion
from sklearn import metrics #Para calcular la tasa de exactitud del modelo
from sklearn.feature_extraction.text import CountVectorizer #Para generar las matrices de palabras
from sklearn.naive_bayes import MultinomialNB #El algoritmo de aprendizaje
from google.colab import drive  #Para montar el Drive
from wordcloud import WordCloud # Para mostrar graficos de las frecuencias de palabras
import matplotlib.pyplot as plt #Para ayudar a WordCloud
import pickle #Para exportar el modelo

# Conexión con Drive

Se establece una conexión con Drive. Para esto deberemos dar acceso a nuestro Drive a Google Colab.

In [ ]:
drive.mount('/content/drive')

# Importación de datos desde Drive

Se leen los datos de la carpeta del Drive. Para esto será necesario tener el acceso directo de la carpeta compartida "Machine Learning TP" en el directorio principal de Drive.

In [ ]:
mail_data = pd.read_csv('/content/drive/My Drive/Machine Learning TP - Llanos-Lopez/Datasets/Mails/mail_data.csv')

# Separación de datos

Como podemos ver, las categorias de la columna Category están representadas por los strings "ham" y "spam", por lo que es necesario convertirlas a un valor numérico para facilitar la resolución.

In [ ]:
mail_data

En esta parte del código, entonces, modificamos a mail_data para que tenga valores numéricos en su categorización. Para los mensajes que son spam, se tendrá como representación al número 1 y para los que no son spam, el número 0. 

In [ ]:
mail_data = pd.get_dummies(data = mail_data, columns=['Category'], drop_first=True)
mail_data

Aca se puede ver como hay 4825 mensajes que no son spam, y 747 que si lo son

In [ ]:
mail_data.groupby("Category_spam").describe()

# Visualización de datos



In [ ]:
#WordCloud para ver palabras frecuentes en los emails ham
spam_words = ' '.join(list(mail_data[mail_data['Category_spam'] == 1]['Message']))
spam_wc = WordCloud(width = 512,height = 512).generate(spam_words)
plt.figure(figsize = (10, 8), facecolor = 'k')
plt.imshow(spam_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

In [ ]:
#WordCloud para ver palabras frecuentes en los emails spam
spam_words = ' '.join(list(mail_data[mail_data['Category_spam'] == 0]['Message']))
spam_wc = WordCloud(width = 512,height = 512).generate(spam_words)
plt.figure(figsize = (10, 8), facecolor = 'k')
plt.imshow(spam_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

Ahora que tenemos los datos como queremos, los separamos para entrenar al modelo. Para el testeo usaremos el %20 de los datos.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mail_data.Message, mail_data.Category_spam, test_size = 0.2, shuffle=False)

En esta parte transformamos los textos a arrays númericos para poder ser usados por el algoritmo MultinomialNB, por lo que ya tendríamos nuestros datos listos: X_train_count y y_train.

In [ ]:
CV = CountVectorizer()
X_train_count = CV.fit_transform(X_train.values)

# Creación y entrenamiento del modelo

Ahora simplemente creamos el modelo con el algoritmo seleccionado y lo entrenamos con los datos seleccionados.

In [ ]:
model = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
model.fit(X_train_count,y_train)

# Evaluación del modelo

En este paso transformamos los X_test y creamos una variable test que tendrá las predicciones hechas por el modelo.

In [ ]:
X_test_count = CV.transform(X_test.values)
score = model.score(X_test_count, y_test)
print(score)

Aqui mostramos la precision del modelo. El valor mostrado puede cambiar al correr el programa multiples veces ya que en la parte de separación de datos se pueden elegir otros valores para testeo y entrenamiento (ya que es una elección aleatoria). En esta demostración, que varia de un rango desde el 0.0 a 1.0, un valor cercano a 1.0 significa una buena precisión.

# Predicción

In [ ]:
def predecir(texto):
  email = [texto]
  texto_count = CV.transform(email)
  resultado = model.predict(texto_count)
  for i in resultado:
    if i == 0:
        print("ham")
    elif i == 1:
        print("spam")


In [ ]:
predecir('Mila, age23, blonde, new in UK. I look sex with UK guys. if u like fun with me. Text MTALK to 69866.18 . 30pp/txt 1st 5free. Â£1.50 increments. Help08718728876')
predecir('Hi. Wk been ok - on hols now! Yes on for a bit of a run. Forgot that i have hairdressers appointment at four so need to get home n shower beforehand. Does that cause prob for u?')

# Exportación de modelo

In [ ]:
#save the count vector file
with open('/content/drive/My Drive/Machine Learning TP - Llanos-Lopez/Modelos/Emails/count_vect.pkl', 'wb') as handle:
    pickle.dump(CV, handle, pickle.HIGHEST_PROTOCOL)

In [ ]:
#save the model file
with open('/content/drive/My Drive/Machine Learning TP - Llanos-Lopez/Modelos/Emails/emails_model.pkl', 'wb') as handle:
    pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)